In [ ]:
import pandas as pd, numpy as np, os
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import matplotlib as plt
import plotly.figure_factory as ff
import plotly.express as px

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier 

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc

from sklearn.linear_model import SGDClassifier
import datatable as dt
import warnings
warnings.filterwarnings('ignore')

SEED = 1
FOLD = 5

In [ ]:
#import files here
%time
train = dt.fread('../input/tsp21sep-data-folder/final_base_train.csv').to_pandas()
test = dt.fread('../input/tsp21sep-data-folder/final_base_test.csv').to_pandas()

sample_submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
#preprocessing
y = train['y_valid']
train.drop('y_valid',inplace=True,axis=1)

X = train
y_meta = y
test_data = test

# Train Data KDE Plot
___

In [ ]:
#thanks to Edrick Kesuma
#use for loop to get below columns
X_data = [X.old_lgbm1, X.old_lgbm3, X.old_catb1, X.old_catb2, X.old_xgb1, X.old_xgb2, X.my_lgbm1, X.my_lgbm2, X.my_lgbm3, X.my_xgb1, X.my_xgb2, X.my_catb1, X.my_catb2, X.my_lgbm4, X.voting_catboost1, X.voting_catboost2, X.voting_xgboost1, X.voting_xgboost2, X.voting_lgbm1, X.voting_lgbm3, X.copy_lgbm2, X.new_lgbm2, X.lgbm1_latest28, X.lgbm1_latest28seed, X.lgbm1_latest28seed2, X.xgb_latest99, X.xgb_latest99_seed, X.lgbm1_gpu, X.lgbm1_cpu, X.colsample_lgbm, X.lgbm2_43gpu, X.lastlgbm43]
group_labels1 = X.columns.to_list()
fig = ff.create_distplot(X_data, group_labels1, bin_size=0.3, show_hist=False, show_rug=False)
fig.show()

# Test Data KDE Plot
___

In [ ]:
_data = [X.old_lgbm1, X.old_lgbm3, X.old_catb1, X.old_catb2, X.old_xgb1, X.old_xgb2, X.my_lgbm1, X.my_lgbm2, X.my_lgbm3, X.my_xgb1, X.my_xgb2, X.my_catb1, X.my_catb2, X.my_lgbm4, X.voting_catboost1, X.voting_catboost2, X.voting_xgboost1, X.voting_xgboost2, X.voting_lgbm1, X.voting_lgbm3, X.copy_lgbm2, X.new_lgbm2, X.lgbm1_latest28, X.lgbm1_latest28seed, X.lgbm1_latest28seed2, X.xgb_latest99, X.xgb_latest99_seed, X.lgbm1_gpu, X.lgbm1_cpu, X.colsample_lgbm, X.lgbm2_43gpu, X.lastlgbm43]
group_labels = test_data.columns.to_list()
fig = ff.create_distplot(_data, group_labels, bin_size=0.3, show_hist=False, show_rug=False)
fig.show()

# Train Data Heatmap
___

In [ ]:
#correlation between all models pred
data = np.corrcoef(X_data)
fig=px.imshow(data,x=group_labels1, y=group_labels1)

fig.show()

# Test Data Heatmap
___

In [ ]:
#correlation between all models pred
data = np.corrcoef(_data)
fig=px.imshow(data,x=group_labels, y=group_labels)

fig.show()

In [ ]:
removed = ['voting_xgboost1', 'old_xgb1', 'my_xgb2']

cols = ['old_lgbm1', 'old_lgbm3', 'old_catb1', 'old_catb2',
       'old_xgb2', 'my_lgbm1', 'my_lgbm2', 'my_lgbm3', 'my_xgb1',
       'my_catb1', 'my_catb2', 'my_lgbm4', 'voting_catboost1',
       'voting_catboost2', 'voting_xgboost2',
       'voting_lgbm1', 'voting_lgbm3', 'copy_lgbm2', 'new_lgbm2',
       'lgbm1_latest28', 'lgbm1_latest28seed', 'lgbm1_latest28seed2',
       'xgb_latest99', 'xgb_latest99_seed', 'lgbm1_gpu', 'lgbm1_cpu',
       'colsample_lgbm', 'lgbm2_43gpu', 'lastlgbm43']

# X = X[cols]
# test_data = test_data[cols]

#check if the all columns in train set is in test set
assert X.columns.to_list() == test_data.columns.to_list()

# Blending | Logistic Regression
___

In [ ]:
# helper functions
def get_auc(y_true, y_hat):
    fpr, tpr, _ = roc_curve(y_true, y_hat)
    score = auc(fpr, tpr)
    return score

In [ ]:
meta_pred_tmp = []
scores_tmp = []

# create cv
kf = StratifiedKFold(n_splits=50, shuffle=True, random_state=1)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    # create train, validation sets
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    model = LogisticRegression(C=4,solver='liblinear',random_state=SEED,n_jobs=-1)
    model.fit(X_train, y_train)
    # validation prediction
    pred_valid = model.predict_proba(X_valid)[:,1]
    
    score = get_auc(y_valid, pred_valid)
    scores_tmp.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('--'*20)
    
    # test prediction based on oof_set
    y_hat = model.predict_proba(test_data)[:,1]
    meta_pred_tmp.append(y_hat)
# print overall validation scores
print(f"Overall Validation Score | Meta: {np.mean(scores_tmp)}")
print('::'*20)

In [ ]:
#average meta predictions over each fold
meta_predictions = np.mean(np.column_stack(meta_pred_tmp), axis=1)

# create submission file
stacked_submission = sample_submission.copy()
stacked_submission['claim'] = meta_predictions
stacked_submission.to_csv('./Logistic_Regression.csv', index=False)

# Blending Prediction Kde Plot
___

In [ ]:
plot = pd.concat([stacked_submission.claim,test_data],axis=1)

pred = [plot.claim, plot.old_lgbm1, plot.old_lgbm3, plot.old_catb1, plot.old_catb2, plot.old_xgb1, plot.old_xgb2, plot.my_lgbm1, plot.my_lgbm2, plot.my_lgbm3, plot.my_xgb1, plot.my_xgb2, plot.my_catb1, plot.my_catb2, plot.my_lgbm4, plot.voting_catboost1, plot.voting_catboost2, plot.voting_xgboost1, plot.voting_xgboost2, plot.voting_lgbm1, plot.voting_lgbm3, plot.copy_lgbm2, plot.new_lgbm2, plot.lgbm1_latest28, plot.lgbm1_latest28seed, plot.lgbm1_latest28seed2, plot.xgb_latest99, plot.xgb_latest99_seed, plot.lgbm1_gpu, plot.lgbm1_cpu, plot.colsample_lgbm, plot.lgbm2_43gpu,plot.lastlgbm43]
group_labels = plot.columns.to_list()
fig = ff.create_distplot(pred, group_labels, bin_size=0.3, show_hist=False, show_rug=False)
fig.show()